# Imports

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

# Read in files

In [2]:
scores = pd.read_csv('data/games_basic.csv')
scores.head(2)

,name,mc_slug,metascore,user_score,date_released
0,Mass Effect 2,mass-effect-2,94,8.9,"Jan 26, 2010"
1,Starcraft II: Wings of Liberty,starcraft-ii-wings-of-liberty,93,8.3,"Jul 27, 2010"


In [3]:
summ = pd.read_csv('data/summary_genres.csv')
summ.head(2)

,mc_slug,summary,genres
0,mass-effect-2,The Mass Effect trilogy is a science fiction a...,"{'Action RPG', 'Role-Playing'}"
1,starcraft-ii-wings-of-liberty,StarCraft II continues the epic saga of the Pr...,"{'Strategy', 'Command', 'Sci-Fi', 'Real-Time'}"


In [4]:
ratings = pd.read_csv('data/ratings.csv')
ratings.head(2)

,mc_slug,rating
0,mass-effect-2,M
1,starcraft-ii-wings-of-liberty,T


In [5]:
descrip = pd.read_csv('data/api_descriptions.csv')
descrip.head(2)

,mc_slug,api_id,description
0,mass-effect-2,4806.0,"Mass Effect II is a sequel to Mass Effect one,..."
1,starcraft-ii-wings-of-liberty,38067.0,StarCraft II: Wings of Liberty is a strategy g...


# Merge data

Since all the dataframes have a common `mc_slug` column, I can merge all the dataframes into one using that column.

In [7]:
games = pd.merge(scores, summ, on='mc_slug').merge(ratings).merge(descrip)
games.head()

,name,mc_slug,metascore,user_score,date_released,summary,genres,rating,api_id,description
0,Mass Effect 2,mass-effect-2,94,8.9,"Jan 26, 2010",The Mass Effect trilogy is a science fiction a...,"{'Action RPG', 'Role-Playing'}",M,4806.0,"Mass Effect II is a sequel to Mass Effect one,..."
1,Starcraft II: Wings of Liberty,starcraft-ii-wings-of-liberty,93,8.3,"Jul 27, 2010",StarCraft II continues the epic saga of the Pr...,"{'Strategy', 'Command', 'Sci-Fi', 'Real-Time'}",T,38067.0,StarCraft II: Wings of Liberty is a strategy g...
2,World of Warcraft: Cataclysm,world-of-warcraft-cataclysm,90,5.7,"Dec 7, 2010","In Cataclysm, players witness the face of Azer...","{'Massively Multiplayer', 'Massively Multiplay...",T,39685.0,World of Warcraft: Cataclysm is a RPG massivel...
3,Sid Meier's Civilization V,sid-meiers-civilization-v,90,8.0,"Sep 21, 2010","With over nine million units sold worldwide, a...","{'Strategy', 'Turn-Based', '4X', 'Historic'}",E10+,13633.0,Real-time strategy genre has always been about...
4,BioShock 2,bioshock-2,88,8.0,"Feb 9, 2010",Set approximately 10 years after the events of...,"{'First-Person', 'Sci-Fi', 'Shooter', 'Action'...",M,4427.0,The second game returns us to the city of Rapt...


In [8]:
# Check to make sure shape is correct
games.shape

(2664, 10)

# Cleaning

Episodic games show up multiple times in the data due to the way they are released. This means that the full game as well as the individual episodes are all listed as separate games in the dataframe. To fix this, redundant episodes will be dropped and/or combined depending on if the full game is also in the dataframe.

## Functions

In [9]:
# Function for getting clean descriptions from RAWG API
def get_clean_descrip(api_id):
    url = f"https://rawg-video-games-database.p.rapidapi.com/games/{int(api_id)}"

    headers = {
        'x-rapidapi-host': "rawg-video-games-database.p.rapidapi.com",
        'x-rapidapi-key': # you api key here
    }

    res = requests.request('GET', url, headers=headers)

    descrip = BeautifulSoup(res.json()['description']).get_text()
    desc_split = descrip.split()
    
    return " ".join(desc_split)


In [10]:
# Scrape summaries from Metacritic
def scrape_summary(game):
    url = f'https://www.metacritic.com/game/pc/{game}'

    user_agent = # your user agent here
    headers = {'User-Agent': user_agent}

    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.content, 'lxml')
 
    summ = soup.find('li', {'class': 'summary_detail product_summary'})

    if summ == None:
        summary = np.nan
    else:
        try:
            summary = summ.find('span', {'class': 'blurb blurb_expanded'}).get_text()
        except:
            summary = summ.get_text()
    
    if len(summary) > 0:
        s = summary.split()

    return ' '.join(s)


In [11]:
# Scrape genres from metacritic
def scrape_genre(game):
    url = f'https://www.metacritic.com/game/pc/{game}'

    user_agent = # your user agent here
    headers = {'User-Agent': user_agent}

    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.content, 'lxml')
    
    genre = soup.find('li', {'class': 'summary_detail product_genre'})
    genre_list = genre.find_all('span', {'class': 'data'})
    genres = set([genre_list[i].text for i in range(len(genre_list))])
    
    return genres

## Dropping Episodic Games

Since I only included PC games released from 2010-2020 that had a metascore, episodic games are usually included multiple times but with different frequencies. Sometimes all the individual episodes and the full game were scored. Most of the time, only a select few of the episodes were scored but not the full game. In this section, I will be dropping the episodes for the games that had their full game scored. 

### The Wolf Among Us (Telltale)

In [12]:
games[games['name'].str.contains('The Wolf Among Us')]

,name,mc_slug,metascore,user_score,date_released,summary,genres,rating,api_id,description
586,The Wolf Among Us: Episode 1 - Faith,the-wolf-among-us-episode-1---faith,85,8.9,"Oct 11, 2013",The Wolf Among Us is a 5-episode series from t...,"{'Point-and-Click', 'General', 'Fantasy', 'Act...",M,NaN,NaN
627,The Wolf Among Us,the-wolf-among-us,80,8.8,"Oct 11, 2013",Based on the award-winning Fables comic book s...,"{'Point-and-Click', 'General', 'Fantasy', 'Act...",M,1682.0,The Wolf Among Us is a five-part episodic game...
866,The Wolf Among Us: Episode 5 - Cry Wolf,the-wolf-among-us-episode-5---cry-wolf,84,8.8,"Jul 8, 2014",Following a bloody trail of murder and corrupt...,"{'Point-and-Click', 'General', 'Fantasy', 'Act...",M,NaN,NaN
879,The Wolf Among Us: Episode 3 - A Crooked Mile,the-wolf-among-us-episode-3---a-crooked-mile,82,8.6,"Apr 8, 2014",After discovering damning evidence at a bloody...,"{'Point-and-Click', 'General', 'Fantasy', 'Act...",M,NaN,NaN
946,The Wolf Among Us: Episode 2 - Smoke and Mirrors,the-wolf-among-us-episode-2---smoke-and-mirrors,76,8.3,"Feb 4, 2014",The Wolf Among Us is a five episode series fro...,"{'Point-and-Click', 'General', 'Fantasy', 'Act...",M,NaN,NaN
958,The Wolf Among Us: Episode 4 - In Sheep's Clot...,the-wolf-among-us-episode-4---in-sheeps-clothing,75,8.2,"May 27, 2014",Summary: The penultimate episode from The Wolf...,"{'Point-and-Click', 'General', 'Fantasy', 'Act...",M,NaN,NaN


In [13]:
# Replace summary of entire game with summary of episode one
games.loc[627, 'summary'] = games['summary'][586]

# Drop all the individual episodes
games.drop(games[games['name'].str.contains('The Wolf Among Us: Ep')].index,
           inplace=True)

In [14]:
games[games['name'].str.contains('The Wolf Among Us')]

,name,mc_slug,metascore,user_score,date_released,summary,genres,rating,api_id,description
627,The Wolf Among Us,the-wolf-among-us,80,8.8,"Oct 11, 2013",The Wolf Among Us is a 5-episode series from t...,"{'Point-and-Click', 'General', 'Fantasy', 'Act...",M,1682.0,The Wolf Among Us is a five-part episodic game...


### The Walking Dead (Telltale)

- The Walking Dead (Season One)
- The Walking Dead: Season Two
- The Walking Dead: Michonne
- The Walking Dead: A New Frontier
- The Walking Dead: The Final Season

### Season 1

In [15]:
# Fill in api_id
games.loc[355, 'api_id'] = 23027

# Get description from api
games.loc[355, 'description'] = get_clean_descrip(23027)

# Drop all the individual episodes
games.drop(games[games['name'].str.contains('The Walking Dead: Ep')].index,
           inplace=True)

### Season 2, Michonne, New Frontier, Final Season

In [16]:
# Drop season 2 episodes
games.drop(games[games['name'].str.contains('The Walking Dead: Season Two Ep')].index,
           inplace=True)

# Drop Michonne episodes
games.drop(games[games['name'].str.contains('Michonne - Ep')].index,
           inplace=True)

# Drop New Frontier episodes
games.drop(games[games['name'].str.contains('A New Frontier Ep')].index,
           inplace=True)

# Drop Final Season episodes
games.drop(games[games['name'].str.contains('The Final Season Ep')].index,
           inplace=True)

### Guardians of the Galaxy (Telltale)

In [17]:
# Drop episodes
games.drop(games[games['name'].str.contains('Galaxy - Ep')].index,
           inplace=True)

### Batman (Telltale)

- Batman: The Telltale Series
- Batman: The Enemy Within

In [18]:
# Drop episodes
games.drop(games[games['name'].str.contains('Batman: The Telltale Series - Ep')].index,
           inplace=True)

# Drop episodes
games.drop(games[games['name'].str.contains('The Enemy Within - Ep')].index,
           inplace=True)

### Tales from the Borderlands (Telltale)

In [19]:
# Drop episodes
games.drop(games[games['name'].str.contains('Borderlands: Ep')].index,
           inplace=True)

### Game of Thrones (Telltale)

In [20]:
# Drop episodes
games.drop(games[games['name'].str.contains('Game of Thrones: Ep')].index,
           inplace=True)

### The Council

In [21]:
# Drop episodes
games.drop(games[games['name'].str.contains('The Council - Ep')].index,
           inplace=True)

### Kentucky Route Zero

In [22]:
# Drop episodes
games.drop(games[games['name'].str.contains('Kentucky Route Zero - Act')].index,
           inplace=True)

### Life is Strange

- Life is Strange
- Life is Strange: Before the Storm
- Life is Strange 2

In [23]:
# Drop Life is Strange episodes
games.drop(games[games['name'].str.contains('Life is Strange: Ep')].index,
           inplace=True)

# Drop Life is Strange: Before the Storm episodes
games.drop(games[games['name'].str.contains('Life is Strange: Before the Storm - Ep')].index,
           inplace=True)

# Drop Life is Strange 2 episodes
games.drop(games[games['name'].str.contains('Life is Strange 2: Ep')].index,
           inplace=True)

### Resident Evil Revelations 2

In [24]:
# Drop episodes
games.drop(games[games['name'].str.contains('Revelations 2 - Ep')].index,
           inplace=True)

### Broken Age

In [25]:
# Drop acts
games.drop(games[games['name'].str.contains('Broken Age: Act')].index,
           inplace=True)

### The Raven: Legacy of a Master Thief

In [26]:
# Drop chapters
games.drop(games[games['name'].str.contains('Master Thief -')].index,
           inplace=True)

### Dreamfall Chapters

In [27]:
# Drop books
games.drop(games[games['name'].str.contains('Dreamfall Chapters Book')].index,
           inplace=True)

## Combining Episodic Games

Not every episode in a game or the full game had a metascore. So for those that only had a few episodes scored but not the full game, I created a new row for the full game with data scraped from Metacritic and dropped the individual episodes. For games that had all the episodes listed but not the full game, I combined them into one.

### Minecraft: Story Mode (Telltale)

- Season 1
- Season 2

In [28]:
# Create the two main games (season 1 and season 2)
# https://www.metacritic.com/game/pc/minecraft-story-mode---a-telltale-games-series---the-complete-adventure
minecraft1 = {
    'name': "Minecraft: Story Mode - A Telltale Games Series - The Complete Edition",
    'mc_slug': 'minecraft-story-mode---a-telltale-games-series---the-complete-adventure',
    'metascore': np.nan,
    'user_score': 6.5,
    'date_released': 'Oct 25, 2016',
    'summary': "Summary: As Jesse, you will embark on a perilous adventure across the Overworld, through the Nether, to the End, and beyond.",
    'genres': {'Adventure', 'Point-and-Click'},
    'rating': 'E10+',
    'api_id': 15916,
    'description': get_clean_descrip(15916)
}

# https://www.metacritic.com/game/pc/minecraft-story-mode---season-two-the-telltale-series
minecraft2 = {
    'name': "Minecraft: Story Mode - Season Two: The Telltale Series",
    'mc_slug': 'minecraft-story-mode---season-two-the-telltale-series',
    'metascore': np.nan,
    'user_score': 3.1,
    'date_released': 'Jul 11, 2017',
    'summary': "Summary: Together with old pals and new comrades alike, Jesse embarks on a brand new journey filled with tough choices, good times, and at least one temperamental llama.",
    'genres': {'Adventure', 'Point-and-Click'},
    'rating': 'E10+',
    'api_id': 28039,
    'description': get_clean_descrip(28039)
}

# Drop episodes
games.drop(games[games['name'].str.contains('Minecraft: Story')].index,
           inplace=True)

# Append main games to df
games = games.append(minecraft1, ignore_index=True)
games = games.append(minecraft2, ignore_index=True)

In [29]:
games.tail(2)

,name,mc_slug,metascore,user_score,date_released,summary,genres,rating,api_id,description
2576,Minecraft: Story Mode - A Telltale Games Serie...,minecraft-story-mode---a-telltale-games-series...,NaN,6.5,"Oct 25, 2016","Summary: As Jesse, you will embark on a perilo...","{Point-and-Click, Adventure}",E10+,15916.0,Minecraft: Story Mode is an action-adventure g...
2577,Minecraft: Story Mode - Season Two: The Tellta...,minecraft-story-mode---season-two-the-telltale...,NaN,3.1,"Jul 11, 2017",Summary: Together with old pals and new comrad...,"{Point-and-Click, Adventure}",E10+,28039.0,Now that Jesse and the gang have vanquished th...


### Sam & Max (Telltale)

In [30]:
# https://www.metacritic.com/game/pc/sam-max-the-devils-playhouse
sam_max = {
    'name': "Sam & Max: The Devil's Playhouse",
    'mc_slug': 'sam-max-the-devils-playhouse',
    'metascore': np.nan,
    'user_score': 7.8,
    'date_released': 'Apr 15, 2010',
    'summary': scrape_summary('sam-max-the-devils-playhouse'),
    'genres': scrape_genre('sam-max-the-devils-playhouse'),
    'rating': np.nan,
    'api_id': np.nan,
    'description': np.nan
}

# Drop episodes
games.drop(games[games['name'].str.contains('Sam & Max')].index,
           inplace=True)

# Append to df
games = games.append(sam_max, ignore_index=True)

### Cognition

In [31]:
# https://www.metacritic.com/game/pc/cognition-an-erica-reed-thriller
cognition = {
    'name': "Cognition: An Erica Reed Thriller",
    'mc_slug': 'cognition-an-erica-reed-thriller',
    'metascore': np.nan,
    'user_score': 8.0,
    'date_released': 'Sep 19, 2013',
    'summary': scrape_summary('cognition-an-erica-reed-thriller'),
    'genres': scrape_genre('cognition-an-erica-reed-thriller'),
    'rating': np.nan,
    'api_id': 17039,
    'description': get_clean_descrip(17039)
}

# Drop episodes
games.drop(games[games['name'].str.contains('Erica Reed Thriller Ep')].index,
           inplace=True)

# Append to df
games = games.append(cognition, ignore_index=True)

### Back to the Future (Telltale)

In [32]:
# https://www.metacritic.com/game/pc/back-to-the-future-the-game
future = {
    'name': "Back to the Future: The Game",
    'mc_slug': 'back-to-the-future-the-game',
    'metascore': np.nan,
    'user_score': 7.3,
    'date_released': 'Dec 23, 2010',
    'summary': scrape_summary('back-to-the-future-the-game'),
    'genres': scrape_genre('back-to-the-future-the-game'),
    'rating': 'T',
    'api_id': 5877,
    'description': get_clean_descrip(5877)
}

# Drop episodes
games.drop(games[games['name'].str.contains('Back to the Future: The Game - Ep')].index,
           inplace=True)

# Append to df
games = games.append(future, ignore_index=True)

### Bear with Me

In [33]:
games[games['name'].str.contains('Bear')]

,name,mc_slug,metascore,user_score,date_released,summary,genres,rating,api_id,description
1590,Bear With Me: Episode 1,bear-with-me-episode-1,68.0,6.1,"Aug 8, 2016",Summary: Bear With Me is an episodic noir adve...,"{'Point-and-Click', 'Adventure'}",NaN,NaN,NaN


In [34]:
# Update existing info with whole game info
# https://www.metacritic.com/game/pc/bear-with-me

games.loc[1590, 'name'] = "Bear With Me"
games.loc[1590, 'mc_slug'] = 'bear-with-me'
games.loc[1590, 'metascore'] = np.nan
games.loc[1590, 'user_score'] = 6.4
games.loc[1590, 'api_id'] = 12738
games.loc[1590, 'description'] = get_clean_descrip(12738)

In [35]:
games[games['name'].str.contains('Bear')]

,name,mc_slug,metascore,user_score,date_released,summary,genres,rating,api_id,description
1590,Bear With Me,bear-with-me,NaN,6.4,"Aug 8, 2016",Summary: Bear With Me is an episodic noir adve...,"{'Point-and-Click', 'Adventure'}",NaN,12738.0,Bear With Me is an episodic noir adventure gam...


### Hector

- No metacritic info for full game, substitute with info from Episode 1
- Average meta and user scores for all 3 episodes

In [36]:
# Will use this to get an averaged metascore and user score
hector_eps = games[games['name'].str.contains('Hector')]

In [37]:
hector_eps

,name,mc_slug,metascore,user_score,date_released,summary,genres,rating,api_id,description
227,Hector: Badge of Carnage - Episode 1: We Negot...,hector-badge-of-carnage---episode-1-we-negotia...,75.0,6.7,"Apr 27, 2011","Summary: Hector: Badge of Carnage, is a lewd, ...","{'General', 'Role-Playing', 'Action Adventure'...",NaN,40756.0,Hector: Badge of Carnage! Episode 1 - We Negot...
237,Hector: Badge of Carnage - Episode 2: Senseles...,hector-badge-of-carnage---episode-2-senseless-...,73.0,5.9,"Aug 25, 2011",Senseless Acts of Justice is a sequel fans wil...,"{'General', 'Action Adventure', 'Adventure', '...",NaN,41040.0,Hector: Badge of Carnage! Episode 2 - Senseles...
251,Hector: Badge of Carnage - Episode 3: Beyond R...,hector-badge-of-carnage---episode-3-beyond-rea...,70.0,6.5,"Sep 22, 2011",NaN,"{'General', 'Action Adventure', 'Adventure', '...",NaN,41041.0,Hector: Badge of Carnage! Episode 3 - Beyond R...


In [38]:
# Update episode one with full game info

games.loc[227, 'name'] = "Hector: Badge of Carnage - Full Series"
games.loc[227, 'metascore'] = round(hector_eps['metascore'].mean())
games.loc[227, 'user_score'] = round(hector_eps['user_score'].mean(), 1)
games.loc[227, 'api_id'] = 18850
games.loc[227, 'description'] = get_clean_descrip(18850)

# Drop episodes
games.drop(games[games['name'].str.contains('Carnage - Ep')].index,
           inplace=True)

In [39]:
games[games['name'].str.contains('Hector')]

,name,mc_slug,metascore,user_score,date_released,summary,genres,rating,api_id,description
227,Hector: Badge of Carnage - Full Series,hector-badge-of-carnage---episode-1-we-negotia...,73.0,6.4,"Apr 27, 2011","Summary: Hector: Badge of Carnage, is a lewd, ...","{'General', 'Role-Playing', 'Action Adventure'...",NaN,18850.0,Hector Badge of Carnage Full Series is a three...


### Republique
- Rename Episode 1 to just Republique.
- Update meta and user score accordingly
- Update description from Steam

In [40]:
# Function for scraping summary descriptions off Steam
def scrape_steam(url):
    user_agent = # your user agent here
    headers = {'User-Agent': user_agent}

    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.content, 'lxml')
 
    summary = soup.find('div', {'class': 'game_description_snippet'}).get_text()
    s = summary.split()

    return " ".join(s)


In [41]:
games[games['name'].str.contains('Republique')]

,name,mc_slug,metascore,user_score,date_released,summary,genres,rating,api_id,description
1156,Republique: Episode 1 - Exordium,republique-episode-1---exordium,77.0,7.5,"Feb 25, 2015",Summary: Republique Remastered is a topical st...,"{'Linear', 'Action Adventure'}",NaN,41405.0,République: Episode 1 - Exordium is an adventu...
1314,Republique: Episode 4 - God's Acre,republique-episode-4---gods-acre,63.0,NaN,"Dec 17, 2015",Summary: You receive a desperate call from Hop...,"{'Linear', 'Action Adventure'}",NaN,NaN,NaN


In [42]:
# https://www.metacritic.com/game/pc/republique-remastered
# https://store.steampowered.com/app/317100/Republique/

games.loc[1156, 'name'] = 'Republique'
games.loc[1156, 'mc_slug'] = 'republique-remastered'
games.loc[1156, 'metascore'] = np.nan
games.loc[1156, 'user_score'] = 6.3
games.loc[1156, 'api_id'] = 496
games.loc[1156, 'description'] = scrape_steam('https://store.steampowered.com/app/317100/Republique/')

# Drop episodes
games.drop(games[games['name'].str.contains('Republique: Ep')].index,
           inplace=True)

In [43]:
games[games['name'].str.contains('Republique')]

,name,mc_slug,metascore,user_score,date_released,summary,genres,rating,api_id,description
1156,Republique,republique-remastered,NaN,6.3,"Feb 25, 2015",Summary: Republique Remastered is a topical st...,"{'Linear', 'Action Adventure'}",NaN,496.0,Help a woman named Hope escape in a thrilling ...


### Penny Arcade

Correct the name, API ID, and description.

In [44]:
games[games['name'].str.contains('Penny')]

,name,mc_slug,metascore,user_score,date_released,summary,genres,rating,api_id,description
411,Penny Arcade Adventures: Episode Three,penny-arcade-adventures-episode-three,77.0,7.3,"Jun 25, 2012",Summary: The Startling Developments Detective ...,"{'Console-style RPG', 'General', 'Japanese-Sty...",NaN,41389.0,Penny Arcade Adventures: On the Rain-Slick Pre...


In [45]:
games.loc[411, 'name'] = "Penny Arcade's On the Rain-Slick Precipice of Darkness 3"
games.loc[411, 'api_id'] = 3043
games.loc[411, 'description'] = get_clean_descrip(3043)

In [46]:
games[games['name'].str.contains('Penny')]

,name,mc_slug,metascore,user_score,date_released,summary,genres,rating,api_id,description
411,Penny Arcade's On the Rain-Slick Precipice of ...,penny-arcade-adventures-episode-three,77.0,7.3,"Jun 25, 2012",Summary: The Startling Developments Detective ...,"{'Console-style RPG', 'General', 'Japanese-Sty...",NaN,3043.0,Penny Arcade's On the Rain-Slick Precipice of ...


## Dropping Obsolete Games

A couple of the games have since been shut down and are no longer playable or purchasable. I will be dropping these games since they bring no value to my recommender system.

In [47]:
# Names of all the games to be dropped
names = ['Scourge', 'Master X', 'Settlers History', 'Global Agenda', 'LEGO Universe',
         'Crimecraft: Bleedout', 'Need for Speed World', 'All Points Bulletin',
         'Elemental: War of Magic', 'Age of Empires Online', 'Battlefield: Play4Free', 'Darkspore',
         'Gods & Heroes: Rome Rising', 'Super Monday Night Combat', 'The Secret World',
         'War of the Roses', "Tom Clancy's Ghost Recon Phantoms", 'Microsoft Flight',
         'Law & Order: Legacies', 'Ace of Spades', 'Infestation: Survivor Stories',
         'War of the Vikings', 'Metro: Last Light - Faction Pack', 'Magicka: Wizard Wars',
         'Infinite Crisis', 'Rise of Incarnates', 'Total War: Arena']

In [48]:
# Grab all the games listed above and drop them by their indices
games.drop(games[games['name'].str.contains('|'.join(names))].index,
           inplace=True)

In [49]:
# Drop Forge
games.drop(games[games['name'] == 'Forge'].index,
           inplace=True)

# Save aggregated games as csv

In [50]:
games.shape

(2536, 10)

In [51]:
games.to_csv('data/cleaned_games.csv', index=False)